In [1]:
# importing the default libraries for data analysis and exploration

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import collections

In [2]:
# downloading the FIFA20 complete player dataset from Kaggle

from google.colab import files
uploaded = files.upload()
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d stefanoleone992/fifa-20-complete-player-dataset

Saving fixture_list.csv to fixture_list.csv
Saving gk_adv.csv to gk_adv.csv
Saving kaggle.json to kaggle.json
Saving league table.csv to league table.csv
Saving pos_stats_1.csv to pos_stats_1.csv
Saving sca.csv to sca.csv
Saving shoo_stats.csv to shoo_stats.csv
-rw-r--r-- 1 root root 67 Oct  4 05:12 kaggle.json
 69% 10.0M/14.5M [00:00<00:00, 38.3MB/s]
100% 14.5M/14.5M [00:00<00:00, 48.2MB/s]


In [3]:
!unzip fifa-20-complete-player-dataset.zip

Archive:  fifa-20-complete-player-dataset.zip
  inflating: players_15.csv          
  inflating: players_16.csv          
  inflating: players_17.csv          
  inflating: players_18.csv          
  inflating: players_19.csv          
  inflating: players_20.csv          
  inflating: teams_and_leagues.csv   


In [4]:
df_15 = pd.read_csv('players_15.csv',error_bad_lines=False)
df_16 = pd.read_csv('players_16.csv',error_bad_lines=False)
df_17 = pd.read_csv('players_17.csv',error_bad_lines=False)
df_18 = pd.read_csv('players_18.csv',error_bad_lines=False)
df_19 = pd.read_csv('players_19.csv',error_bad_lines=False)
df_20 = pd.read_csv('players_20.csv',error_bad_lines=False)


In [5]:
# checking the initial dataset
df_20.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,NaN,NaN,NaN,...,48,40,94,94,75,96,33,37,26,6,11,15,14,8,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,NaN,NaN,NaN,...,63,29,95,82,85,95,28,32,24,7,11,15,14,11,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,2017-08-03,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,NaN,NaN,NaN,...,51,36,87,90,90,94,27,26,29,9,9,15,15,11,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,90+3,90+3,90+3,89+3,82+3,82+3,82+3,89+3,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,2014-07-16,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,92.0,78.0,...,34,19,11,65,11,68,27,12,18,87,92,78,90,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,2019-07-01,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,NaN,NaN,NaN,...,54,41,87,89,88,91,34,27,22,11,12,6,8,8,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


In [6]:
# dropping the SOFIFA ID (unique identifier used by FIFA)
df_20 = df_20.drop(['sofifa_id','player_url'],axis = 1)

In [7]:
# Complete list of the columns of the dataset
print(list(df_20.columns))

['short_name', 'long_name', 'age', 'dob', 'height_cm', 'weight_kg', 'nationality', 'club', 'overall', 'potential', 'value_eur', 'wage_eur', 'player_positions', 'preferred_foot', 'international_reputation', 'weak_foot', 'skill_moves', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'team_position', 'team_jersey_number', 'loaned_from', 'joined', 'contract_valid_until', 'nation_position', 'nation_jersey_number', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'player_traits', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 

In [8]:
# checking what all columns have NaN values
df_20.isnull().sum()

short_name       0
long_name        0
age              0
dob              0
height_cm        0
              ... 
lb            2036
lcb           2036
cb            2036
rcb           2036
rb            2036
Length: 102, dtype: int64

In [10]:
columns = [ 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb']
df_20.loc[:,columns]

,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,90+3,90+3,90+3,89+3,82+3,82+3,82+3,89+3,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18273,32+2,32+2,32+2,31+2,31+2,31+2,31+2,31+2,31+2,31+2,31+2,33+2,33+2,33+2,33+2,33+2,43+2,42+2,42+2,42+2,43+2,45+2,46+2,46+2,46+2,45+2
18274,33+2,33+2,33+2,33+2,32+2,32+2,32+2,33+2,33+2,33+2,33+2,35+2,36+2,36+2,36+2,35+2,44+2,43+2,43+2,43+2,44+2,46+2,47+2,47+2,47+2,46+2
18275,43+2,43+2,43+2,43+2,43+2,43+2,43+2,43+2,44+2,44+2,44+2,45+2,46+2,46+2,46+2,45+2,47+2,49+2,49+2,49+2,47+2,47+2,49+2,49+2,49+2,47+2
18276,43+2,43+2,43+2,45+2,44+2,44+2,44+2,45+2,46+2,46+2,46+2,46+2,46+2,46+2,46+2,46+2,48+2,48+2,48+2,48+2,48+2,48+2,49+2,49+2,49+2,48+2


In [11]:
# splitting the column of positional rating that come equipped with the chemistry/form boost
df_20['chem_boost'] = df_20['ls'].str.split('+',n=1,expand = True)[1]
for col in columns:
  df_20[col] = df_20[col].str.split('+',n=1,expand = True)[0]

In [13]:
# trying a simple function to call the team based on the Club

def get_club_data(name):
  team = df_20.loc[df_20['club'] == name]
  team['team_jersey_number'] = [int(x) for x in team['team_jersey_number']]
  #team = team.set_index('team_jersey_number')
  return team
mufc = get_club_data('Manchester United')

mufc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,...,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,chem_boost
14,De Gea,David De Gea Quintana,28,1990-11-07,192,82,Spain,Manchester United,89,90,56000000,205000,GK,Right,4,3,1,Medium/Medium,Lean,Yes,110600000.0,NaN,GK,1,NaN,2011-07-01,2020.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,90.0,84.0,81.0,92.0,58.0,...,30,12,65,29,68,25,21,13,90,84,81,85,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,P. Pogba,Paul Pogba,26,1993-03-15,191,84,France,Manchester United,88,91,72500000,250000,"CM, CDM",Right,4,4,5,High/Medium,Normal,Yes,143200000.0,"#Dribbler, #Playmaker , #Strength, #Complete ...",LDM,6,NaN,2016-08-09,2021.0,RDM,6.0,74.0,81.0,86.0,85.0,66.0,86.0,NaN,NaN,NaN,NaN,NaN,...,64,83,88,83,87,63,67,65,5,6,2,4,3,81,81,81,82,83,83,83,82,84,84,84,83,84,84,84,83,76,77,77,77,76,74,72,72,72,74,3
163,A. Martial,Anthony Martial,23,1995-12-05,184,76,France,Manchester United,83,88,34500000,140000,"LW, ST, LM",Right,3,3,4,Medium/Low,Normal,Yes,68100000.0,#Dribbler,ST,9,NaN,2015-09-01,2024.0,SUB,19.0,89.0,81.0,72.0,86.0,41.0,71.0,NaN,NaN,NaN,NaN,NaN,...,42,82,73,81,79,34,39,36,9,8,8,15,11,81,81,81,82,82,82,82,82,80,80,80,80,73,73,73,80,64,59,59,59,64,60,54,54,54,60,3
171,M. Rashford,Marcus Rashford,21,1997-10-31,186,70,England,Manchester United,83,88,35500000,125000,"ST, LM",Right,2,3,5,High/High,Lean,Yes,70100000.0,"#Speedster, #Engine, #Acrobat",LM,10,NaN,2014-07-01,2023.0,SUB,11.0,92.0,82.0,73.0,84.0,45.0,77.0,NaN,NaN,NaN,NaN,NaN,...,42,84,75,76,78,47,40,33,11,6,15,7,14,81,81,81,82,82,82,82,82,80,80,80,81,73,73,73,81,65,62,62,62,65,62,57,57,57,62,2
280,H. Maguire,Harry Maguire,26,1993-03-05,194,100,England,Manchester United,82,83,23000000,125000,CB,Right,1,3,2,High/High,Stocky,Yes,45400000.0,"#Aerial Threat, #Strength",LCB,5,NaN,2019-08-05,2024.0,LCB,6.0,50.0,53.0,64.0,69.0,81.0,84.0,NaN,NaN,NaN,NaN,NaN,...,78,61,66,44,82,81,83,79,14,16,9,14,9,66,66,66,62,65,65,65,62,66,66,66,64,71,71,71,64,70,78,78,78,70,72,81,81,81,72,2
295,Juan Mata,Juan Manuel Mata García,31,1988-04-28,170,63,Spain,Manchester United,82,82,19500000,135000,"CAM, RM",Left,3,3,4,Medium/Medium,Lean,Yes,37100000.0,#FK Specialist,SUB,8,NaN,2014-01-25,2021.0,NaN,NaN,62.0,74.0,83.0,83.0,36.0,47.0,NaN,NaN,NaN,NaN,NaN,...,39,81,84,80,87,38,31,21,9,10,14,5,8,72,72,72,78,78,78,78,78,80,80,80,79,76,76,76,79,60,59,59,59,60,55,45,45,45,55,3
301,A. Sánchez,Alexis Alejandro Sánchez Sánchez,30,1988-12-19,169,62,Chile,Manchester United,82,82,21000000,150000,"LW, LM",Right,4,3,4,High/High,Normal,Yes,39900000.0,NaN,SUB,7,NaN,2018-01-22,2022.0,LW,7.0,81.0,78.0,77.0,85.0,44.0,73.0,NaN,NaN,NaN,NaN,NaN,...,46,82,79,74,79,38,44,35,10,10,15,12,13,78,78,78,80,80,80,80,80,80,80,80,79,75,75,75,79,63,62,62,62,63,60,56,56,56,60,3
319,J. Lingard,Jesse Lingard,26,1992-12-15,175,62,England,Manchester United,82,82,24000000,135000,"CAM, RW",Right,2,3,4,High/High,Lean,Yes,45600000.0,"#Engine, #Acrobat",CAM,14,NaN,2011-07-01,2021.0,SUB,7.0,80.0,78.0,76.0,83.0,58.0,72.0,NaN,NaN,NaN,NaN,NaN,...,54,84,81,74,82,63,57,49,14,10,8,13,9,78,78,78,80,81,81,81,80,80,80,80,80,77,77,77,80,71,68,68,68,71,68,63,63,63,68,2
335,V. Lindelöf,Victor Nilsson Lindelöf,24,1994-07-17,187,80,Sweden,Manchester United,

In [15]:
#   In this cell we assign values to Goalkeepers for other positions
#   Needless to say goalkeepers will not have near enough proficiency
#   in those positions as they are goalkeepers. That being said they
#   aren't completely incompetent, therefore we assign them minimal 
#   values in those position based on their strong foot.

#   We also create a new position called Goalkeeper in the columns 
#   so that we can assign the goalkeeping overall and access it 
#   directly

lcols = ['ls','lf','lw','lam','lm','lb','lwb']
rcols = ['rs','rf','rw','ram','rm','rb','rwb']
ccols = ['st','cm','cam','cdm','cb','lcb','rcb','cf','lcm','rcm','ldm','rdm']
for i in range(len(df_20)):
  if df_20.loc[i,'player_positions'] == 'GK':
    df_20.loc[i,'gk'] = df_20.loc[i,'overall']
    if df_20.loc[i,'preferred_foot'] == 'Right':
      for col in rcols:
        df_20.loc[i,col] = 45
      for col in lcols:
        df_20.loc[i,col] = 35
      for col in ccols:
        df_20.loc[i,col] = 40
    else:
      for col in lcols:
        df_20.loc[i,col] = 45
      for col in rcols:
        df_20.loc[i,col] = 35
      for col in ccols:
        df_20.loc[i,col] = 40
  else:
    df_20.loc[i,'gk'] = int(df_20.loc[i,'overall']*0.4)
#print(df_20['gk'])
df_20['chem_boost'] = df_20['chem_boost'].fillna(3)

In [19]:
#   Converting the column data types to integer type
int_conv = ['gk', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb','chem_boost']
for col in int_conv:
  df_20[col] = [int(x) for x in df_20[col]]
#print(df_20.dtypes[0:50])
columns = [ 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb']

# Given some glaring discrepancy in the FIFA 20 Dataset, (particularly in Dortmund's overall), 
# I took it open myself to make things closer to the ground truth.

# PS: United fans you'll notice I did not change Jadon Sancho's club to Manchester United
# because he's not leaving. Get over it. 
for col in columns:
  df_20.loc[df_20[df_20['club'] == 'Leeds United'].index.values[0]][col] += 5
#df_20[df_20['long_name'] == 'Erling Braut Håland']
df_20.loc[2655,'club'] = 'Borussia Dortmund'
df_20.loc[2655,'age'] = 20
df_20.loc[2655,'team_jersey_number'] = 9.0
df_20.loc[2655,'overall'] = 84
df_20.loc[2655,'pace'] = 84
df_20.loc[2655,'shooting'] = 87
df_20.loc[2655,'passing'] = 63
df_20.loc[2655,'dribbling'] = 76
df_20.loc[2655,'defending'] = 43
df_20.loc[2655,'physic'] = 85
df_20.loc[2655,'st'] = 84
df_20.loc[2655,'ls'] = 84
df_20.loc[2655,'rs'] = 84
df_20.loc[108,'overall'] = 87
df_20.loc[108,'rm'] = 87
df_20.loc[108,'lm'] = 87
df_20.loc[108,'rw'] = 87
#df_20[df_20['long_name'] == 'Jadon Sancho'].iloc[0:50]
df_20.loc[186,'club'] = 'Villarreal CF'
#df_20[df_20['short_name']== 'Paco Alcácer']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# checking for NaN values again to verify that all NaN values relevant to the dataset have been eliminated
print(df_20.isnull().sum())

short_name    0
long_name     0
age           0
dob           0
height_cm     0
             ..
cb            0
rcb           0
rb            0
chem_boost    0
gk            0
Length: 104, dtype: int64


In [ ]:
"""

The next blob of code contains the class that we shall be using for the majority
of the project. We make a class that can be used to contain the information of 
one club and enable easy and seamless interaction between clubs. 

As described we use the get_club_data function (albeit a little modified) to 
fetch the player dataset for the particular player. For stastical and the
mathematical modelling of a "club" I felt that getting as many variables describing
the club would make the process more detailed and therefore more accurate. 

In this project a club can be viewed as a collection of variables. The club's 
entire overall, the club's defensive overall, midfield overall, attack overall,
the standalone goalkeeper statistics and overall, and the sub's bench. 

Keep in mind that in a game of football the only utilities you have are the 
eleven starting players, and the seven subs out of which only 3 are used. At 
the time of creation however due to COVID the 5 subs rule was in place and 
was therefore incorporated in the model. 

The overall of the team or club therefore depends on the players on the pitch. 

And no, Barca fans, you can't expect one 93 rated Lionel Messi to 
carry you to glory. 

Since this is a mathematical model the "history" of the club is irrelevant, 
as it should be. 
"""

In [25]:
class club:

  def __init__(self,team_name):
    self.team_name = team_name
    self.starting_eleven = collections.OrderedDict()
    self.defense = collections.OrderedDict()
    self.midfield = collections.OrderedDict()
    self.attack = collections.OrderedDict()
    self.goalkeep = collections.OrderedDict()
    self.sub_5 = collections.OrderedDict()
    self.team_data = df_20.loc[:,:]
    self.ovr = 0
    self.get_team_data()
    self.get_ideal_line()
    #self.avg_possession = p_stats[p_stats['Squad'] == self.team_name]['Poss'].values[0]


  def get_ideal_line(self):
    '''
    This function is an approximation of a way to select the best lineup 
    for a team. Pick the most high rated players for a particular position
    for that position. 

    While this is a function called by default, the model is also open
    (and encouragingly so) for user inputs of the team the user wishes to field.
    '''
    copy = self.team_data
    gcolumns = ['gk']
    dcolumns = ['cb','cb','lb','rb']
    mcolumns = ['cam','cm','cm']
    acolumns = ['lw','st','rw']
    columns = ['gk','cb','cb','lb','rb','cam','cm','cm','lw','st','rw']
    

    
    for col in gcolumns:
      self.goalkeep[copy[copy[col] == copy[col].max()]['team_jersey_number'].values[0]] = col
    for col in dcolumns:
      self.defense[copy[copy[col] == copy[col].max()]['team_jersey_number'].values[0]] = col
      copy = copy.drop(copy[col].idxmax(),axis = 0)
    for col in mcolumns:
      self.midfield[copy[copy[col] == copy[col].max()]['team_jersey_number'].values[0]] = col
      copy = copy.drop(copy[col].idxmax(),axis = 0)
    for col in acolumns:
      self.attack[copy[copy[col] == copy[col].max()]['team_jersey_number'].values[0]] = col
      copy = copy.drop(copy[col].idxmax(),axis = 0)
    
    self.starting_eleven = {**self.goalkeep,**self.defense,**self.midfield,**self.attack}
    
    for i in range(5):
      self.sub_5[copy[copy['overall'] == copy['overall'].max()]['team_jersey_number'].values[0]] = 'SUB'
      copy = copy.drop(copy['overall'].idxmax(),axis = 0)
    
 
  def get_team_data(self):
    self.team_data = df_20[df_20['club']==self.team_name]
    #self.team_data.loc['team_jersey_number'] = [int(x) for x in self.team_data['team_jersey_number']]
  
  def print_starters(self):

    # Simple function for printing the starting lineup with positions 
    for x,y in zip(self.starting_eleven,self.starting_eleven.values()):
      print('{} : {}'.format(y.upper(),self.team_data[self.team_data['team_jersey_number'] == x]['short_name'].values[0]) )

  def get_ovr(self,goalkeep,defense,midfield,attack,starting_eleven,sub,form):
    '''
    One of the driver functions behind the model, the get_ovr function 
    aims to use the positional overall and the base overall of each and every
    player to get the variables of the class described in the previous blob

    The class has 5 main values that it judges the team upon : 
    1) Goalkeeper Overall
    2) Aggregate Defense Overall
    3) Aggregate Midfield Overall
    4) Aggregate Attack Overall
    5) Aggregate Substitute Overall
    6) The form of the team (upto three previous matches)
    '''
    pos = []
    jernum = []
    self.ovr = 0
    self.mid_ovr = 0
    self.def_ovr = 0
    self.att_ovr = 0
    self.gk_ovr = 0
    self.lineup = pd.DataFrame()
    self.midfield = pd.DataFrame()
    self.attack = pd.DataFrame()
    self.goalkeep = pd.DataFrame()
    self.defense = pd.DataFrame()
    self.form = form
    for x,y in zip(midfield,midfield.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.midfield = self.midfield.append(self.team_data[self.team_data['team_jersey_number'] == x])
      self.mid_ovr += a[y].values[0]
    self.mid_ovr = (self.mid_ovr)/len(midfield)
    
    min_def = 0
    for x,y in zip(defense,defense.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.defense = self.defense.append(self.team_data[self.team_data['team_jersey_number'] == x])
      self.def_ovr += a[y].values[0]
      if min_def == 0:
        min_def = a[y].values[0]
      elif min_def<a[y].values[0]:
        min_def = a[y].values[0]
    self.def_ovr = ((self.def_ovr/len(defense)) + min_def)/2
    
    for x,y in zip(attack,attack.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.attack = self.attack.append(self.team_data[self.team_data['team_jersey_number'] == x])
      self.att_ovr += a[y].values[0]
    self.att_ovr = self.att_ovr/len(attack)
    
    for x,y in zip(goalkeep,goalkeep.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.goalkeep = self.goalkeep.append(self.team_data[self.team_data['team_jersey_number'] == x])
      self.gk_ovr += a[y].values[0]
    
    for x,y in zip(starting_eleven,starting_eleven.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.lineup = self.lineup.append(self.team_data[self.team_data['team_jersey_number'] == x])
      self.ovr += a[y].values[0]
    
    self.ovr = (self.gk_ovr + self.def_ovr + self.mid_ovr + self.att_ovr)/4
    sub_ovr = 0
    for x,y in zip(sub,sub.values()):
      a = self.team_data[self.team_data['team_jersey_number']==x]
      self.lineup = self.lineup.append(self.team_data[self.team_data['team_jersey_number'] == x])
      sub_ovr += a['overall'].values[0]
    sub_ovr = (sub_ovr*3)/7
    pos = pd.DataFrame([x.upper() for x in list(starting_eleven.values())],index = [x for x in starting_eleven])
    pos_sub = pd.DataFrame([x.upper() for x in list(sub.values())],index = [x for x in sub])
    pos = pos.append(pos_sub)
    self.lineup['Index'] = self.lineup['team_jersey_number']
    self.lineup = self.lineup.set_index('Index')
    self.lineup = self.lineup.loc[:,('short_name','age','overall')]
    self.lineup = pd.concat([self.lineup,pos],axis = 1)
    self.lineup.columns = ['short_name','age','overall','Positions']
    self.ovr += sub_ovr
    return self.ovr

In [26]:
#   Now we try to see if the ideal line-up algorithm works to put the best players
#   available to Borussia Dortmund on the pitch. 
BVB09 = club('Borussia Dortmund')
BVB09.get_ovr(BVB09.goalkeep,BVB09.defense,BVB09.midfield,BVB09.attack,BVB09.starting_eleven,BVB09.sub_5,['W','W','W'])
BVB09.print_starters()

GK : R. Bürki
CB : M. Hummels
CB : A. Witsel
LB : T. Delaney
RB : L. Piszczek
CAM : M. Reus
CM : M. Götze
CM : R. Guerreiro
LW : J. Sancho
ST : E. Håland
RW : J. Brandt


In [ ]:
"""
FIFA however, can only do so much in reference to the representation of the team.
Therefore to simulate interactions between the teams we need statistics that help
us better gauge the team as a whole.

For the same, I have used the amazing site FBRef to obtain squad based statistics
of all the teams in the Big 5 leagues, that is the Italian Serie-A, 
the German Bundesliga,the French Ligue One, the Spanish La Liga, and of course 
the best of the best, the English Premier League

We aim to obtain the stats for Squad Possession, Advanced Goalkeeping statistics,
the Shooting and SoT statistics, and the Shot Creating Actions and 
Chance Creation statistics. 

These statistics are important because Football isn't a game of who's the better
team, it's a game of who scores more goals. Plain and Simple. 

"""

In [27]:
# Reading the statistics from the csv files
p_stats = pd.read_csv('pos_stats_1.csv',error_bad_lines=False,encoding = "ISO-8859-1")
gk_adv = pd.read_csv('gk_adv.csv',error_bad_lines= False,encoding= "ISO-8859-1")
shooting = pd.read_csv('shoo_stats.csv',error_bad_lines=False,encoding = "ISO-8859-1")
sca = pd.read_csv('sca.csv',error_bad_lines=False,encoding = "ISO-8859-1")
p_stats

,Squad,Comp,# Pl,Poss,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Succ,Att,Succ%,#Pl,Megs,Carries,TotDist,PrgDist,Targ,Rec,Rec%,Miscon,Dispos
0,Alavés,es La Liga,32,41.1,17992,2141,6134,8850,3878,563,16042,250,466,53.6,277,14,9417,53334,26549,12391,9724,78.5,458,386
1,Amiens,fr Ligue 1,27,45.5,15903,1660,4991,8238,3582,415,14501,283,476,59.5,317,20,9853,53381,28763,11622,9628,82.8,409,426
2,Angers,fr Ligue 1,26,45.1,16741,1449,5353,8130,4353,594,15332,344,558,61.6,367,26,10806,62206,34355,12508,10616,84.9,386,358
3,Arsenal,eng Premier League,29,54.0,24196,2929,8623,11508,5691,931,22390,428,711,60.2,455,34,16309,97513,55374,18851,16349,86.7,448,451
4,Aston Villa,eng Premier League,28,43.9,19826,2772,7317,8474,5327,780,17901,368,584,63.0,404,25,11555,74828,39599,14087,11530,81.8,409,428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Werder Bremen,de Bundesliga,30,48.8,20863,2519,7265,10087,4783,704,19078,278,487,57.1,296,17,12938,73123,39211,15464,12978,83.9,453,403
97,West Brom,eng Premier League,17,29.7,1336,219,554,565,282,36,1214,19,36,52.8,20,1,681,4004,2097,824,706,85.7,31,40
98,West Ham,eng Premier League,29,44.0,20825,2288,6477,9965,5580,823,19019,406,645,62.9,429,23,12519,76114,39304,15315,12388,80.9,399,408
99,Wolfsburg,de Bundesliga,26,48.3,19911,2102,6576,9616,4946,787,18132,388,617,62.9,414,30,11921,71694,40133,14309,11716,81.9,469,469


In [28]:
gk_adv

,Squad,Comp,# Pl,90s,GA,PKA,FK,CK,OG,PSxG,PSxG/SoT,PSxG+/-,/90,Cmp,Att,Cmp%,Att.1,Thr,Launch%,AvgLen,Att.2,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,Alavés,es La Liga,3,38.0,59,12,3,3,0,57.0,0.32,-2.0,-0.05,401,881,45.5,787,110,71.5,50.4,348,91.4,62.5,356,22,6.2,18,0.47,14.0
1,Amiens,fr Ligue 1,1,28.0,50,1,1,13,1,41.1,0.29,-7.9,-0.28,231,521,44.3,567,95,55.7,43.6,254,80.7,55.4,269,15,5.6,11,0.39,13.7
2,Angers,fr Ligue 1,2,28.0,33,3,1,1,1,25.8,0.27,-6.2,-0.22,162,412,39.3,528,124,47.3,39.8,222,73.0,52.4,215,17,7.9,22,0.79,14.8
3,Arsenal,eng Premier League,2,38.0,48,7,0,9,0,58.0,0.27,10.0,0.26,128,433,29.6,971,220,29.7,32.7,312,46.5,41.4,353,23,6.5,21,0.55,13.3
4,Aston Villa,eng Premier League,4,38.0,67,5,1,10,1,59.8,0.32,-6.2,-0.16,279,723,38.6,850,170,55.2,44.4,343,74.1,55.5,391,26,6.6,19,0.50,13.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Werder Bremen,de Bundesliga,2,34.0,69,1,1,13,3,50.1,0.28,-15.9,-0.47,254,598,42.5,929,170,45.9,39.3,305,56.4,44.8,375,14,3.7,24,0.71,14.3
97,West Brom,eng Premier League,1,3.0,11,2,0,3,0,10.5,0.42,-0.5,-0.18,23,69,33.3,70,11,72.9,51.3,21,85.7,55.8,28,1,3.6,0,0.00,13.3
98,West Ham,eng Premier League,4,38.0,62,5,2,7,3,58.7,0.30,-0.3,-0.01,326,815,40.0,776,129,70.1,50.4,292,92.8,64.5,372,16,4.3,12,0.32,11.7
99,Wolfsburg,de Bundesliga,2,34.0,46,5,1,5,0,45.9,0.31,-0.1,0.00,281,619,45.4,1012,161,47.4,40.0,240,57.9,48.1,290,21,7.2,30,0.88,15.4


In [29]:
shooting

,Squad,Comp,# Pl,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,Alavés,es La Liga,32,34,299,99,33.1,7.87,2.61,0.09,0.28,10.0,6,6,36.9,32.4,0.11,-2.9,-4.4
1,Amiens,fr Ligue 1,27,31,229,77,33.6,8.18,2.75,0.12,0.36,13.0,3,5,26.8,22.9,0.10,4.2,5.1
2,Angers,fr Ligue 1,26,27,367,112,30.5,13.11,4.00,0.07,0.23,16.0,1,1,27.1,26.4,0.07,-0.1,-0.4
3,Arsenal,eng Premier League,29,56,401,144,35.9,10.55,3.79,0.13,0.37,19.0,3,3,49.2,46.9,0.12,6.8,6.1
4,Aston Villa,eng Premier League,28,40,453,146,32.2,11.92,3.84,0.09,0.27,17.0,1,3,40.0,37.6,0.08,0.0,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Werder Bremen,de Bundesliga,30,40,419,135,32.2,12.32,3.97,0.09,0.27,10.0,3,6,41.4,36.9,0.09,-1.4,0.1
97,West Brom,eng Premier League,17,5,22,8,36.4,7.33,2.67,0.23,0.63,2.0,0,0,1.7,1.7,0.08,3.3,3.3
98,West Ham,eng Premier League,29,49,414,157,37.9,10.89,4.13,0.11,0.29,13.0,4,4,46.2,43.2,0.11,2.8,1.8
99,Wolfsburg,de Bundesliga,26,48,466,155,33.3,13.71,4.56,0.09,0.28,11.0,4,4,47.3,44.3,0.10,0.7,-0.3


In [30]:
sca

,Squad,Comp,# Pl,SCA,SCA90,PassLive,PassDead,Drib,Sh,Fld,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,OG
0,Alavés,es La Liga,32,454,11.95,295,72,20,27,40,52,1.37,31,7,1,6,7,0
1,Amiens,fr Ligue 1,27,351,12.54,256,27,28,11,29,50,1.79,34,0,6,4,6,0
2,Angers,fr Ligue 1,26,544,19.43,400,57,35,23,28,48,1.71,36,2,3,3,3,1
3,Arsenal,eng Premier League,29,615,16.18,436,57,50,36,36,78,2.05,57,4,4,9,4,0
4,Aston Villa,eng Premier League,28,747,19.66,549,80,36,27,54,67,1.76,46,7,5,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Werder Bremen,de Bundesliga,30,647,19.03,467,62,42,42,32,68,2.00,41,5,4,11,5,2
97,West Brom,eng Premier League,17,31,10.33,26,1,2,0,2,6,2.00,3,1,1,0,1,0
98,West Ham,eng Premier League,29,653,17.18,471,73,40,28,41,82,2.16,54,7,6,6,9,0
99,Wolfsburg,de Bundesliga,26,699,20.56,482,82,50,45,40,76,2.24,48,5,8,7,8,0


In [ ]:
'''
As we can see, the names of the Squads aren't exactly the same as the ones in
the FIFA Dataset. Therefore we make a dictionary that can ensure the 
seamless conversion of these Squad Names to standard FIFA club names.


'''

In [31]:
possession_new = p_stats 
club_names = list(np.unique(df_20['club']))
pos_to_df = {}
for i in range(len(possession_new)):
  for j in range(len(club_names)):
    if(p_stats.loc[i,'Squad'] in club_names[j]):
      pos_to_df[possession_new.loc[i,'Squad']] = club_names[j]
      possession_new.loc[i,'Squad'] = club_names[j]


In [35]:
print('Clubs in the dictionary : {}'.format(len(pos_to_df)))
print('Clubs in the top 5 leagues : {}'.format(len(p_stats)))
pos_to_df

Clubs in the dictionary : 91
Clubs in the top 5 leagues : 101


{'Alavés': 'Deportivo Alavés',
 'Amiens': 'Amiens SC',
 'Angers': 'Angers SCO',
 'Arsenal': 'Arsenal de Sarandí',
 'Aston Villa': 'Aston Villa',
 'Atalanta': 'Atalanta',
 'Athletic Club': 'Athletic Club de Bilbao',
 'Atlético Madrid': 'Atlético Madrid',
 'Augsburg': 'FC Augsburg',
 'Barcelona': 'FC Barcelona',
 'Betis': 'Real Betis',
 'Bologna': 'Bologna',
 'Bordeaux': 'FC Girondins de Bordeaux',
 'Bournemouth': 'Bournemouth',
 'Brescia': 'Brescia',
 'Brest': 'Stade Brestois 29',
 'Brighton': 'Brighton & Hove Albion',
 'Burnley': 'Burnley',
 'Cagliari': 'Cagliari',
 'Chelsea': 'Chelsea',
 'Crystal Palace': 'Crystal Palace',
 'Dijon': 'Dijon FCO',
 'Dortmund': 'Borussia Dortmund',
 'Düsseldorf': 'Fortuna Düsseldorf',
 'Eibar': 'SD Eibar',
 'Espanyol': 'RCD Espanyol',
 'Everton': 'CD Everton de Viña del Mar',
 'Fiorentina': 'Fiorentina',
 'Freiburg': 'SC Freiburg',
 'Fulham': 'Fulham',
 'Genoa': 'Genoa',
 'Getafe': 'Getafe CF',
 'Granada': 'Granada CF',
 'Hellas Verona': 'Hellas Verona',

In [36]:
# Corrective labelling to include missing clubs and mislabelled clubs
pos_to_df['Arsenal'] = 'Arsenal'
pos_to_df['Everton'] = 'Everton'
pos_to_df['Roma'] = 'Roma'
pos_to_df['Inter'] = 'Inter'
pos_to_df['Manchester Utd'] = 'Manchester United'
pos_to_df['M\'Gladbach'] = 'Borussia Mönchengladbach'
pos_to_df['Bayern Munich'] = 'FC Bayern München'
pos_to_df['Sheffield Utd'] = 'Sheffield United'
pos_to_df['Rennes'] = 'Stade Rennais FC'
pos_to_df['Paris S-G'] = 'Paris Saint-Germain'
pos_to_df['Wolves'] = 'Wolverhampton Wanderers'
pos_to_df['Brest'] = 'Stade Brestois 29'
pos_to_df['Celta Vigo'] = 'RC Celta'
pos_to_df['Eint Frankfurt'] = 'Eintracht Frankfurt'
pos_to_df['Newcastle Utd'] = 'Newcastle United'

In [37]:
pos_to_df

{'Alavés': 'Deportivo Alavés',
 'Amiens': 'Amiens SC',
 'Angers': 'Angers SCO',
 'Arsenal': 'Arsenal',
 'Aston Villa': 'Aston Villa',
 'Atalanta': 'Atalanta',
 'Athletic Club': 'Athletic Club de Bilbao',
 'Atlético Madrid': 'Atlético Madrid',
 'Augsburg': 'FC Augsburg',
 'Barcelona': 'FC Barcelona',
 'Bayern Munich': 'FC Bayern München',
 'Betis': 'Real Betis',
 'Bologna': 'Bologna',
 'Bordeaux': 'FC Girondins de Bordeaux',
 'Bournemouth': 'Bournemouth',
 'Brescia': 'Brescia',
 'Brest': 'Stade Brestois 29',
 'Brighton': 'Brighton & Hove Albion',
 'Burnley': 'Burnley',
 'Cagliari': 'Cagliari',
 'Celta Vigo': 'RC Celta',
 'Chelsea': 'Chelsea',
 'Crystal Palace': 'Crystal Palace',
 'Dijon': 'Dijon FCO',
 'Dortmund': 'Borussia Dortmund',
 'Düsseldorf': 'Fortuna Düsseldorf',
 'Eibar': 'SD Eibar',
 'Eint Frankfurt': 'Eintracht Frankfurt',
 'Espanyol': 'RCD Espanyol',
 'Everton': 'Everton',
 'Fiorentina': 'Fiorentina',
 'Freiburg': 'SC Freiburg',
 'Fulham': 'Fulham',
 'Genoa': 'Genoa',
 'Geta

In [38]:
# converting the squad names to correct FIFA format
p_stats = pd.read_csv('pos_stats_1.csv',error_bad_lines=False,encoding = "ISO-8859-1")
for i in range(len(p_stats)):
  p_stats.loc[i,'Squad'] = pos_to_df[p_stats.loc[i,'Squad']]
p_stats

,Squad,Comp,# Pl,Poss,Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Live,Succ,Att,Succ%,#Pl,Megs,Carries,TotDist,PrgDist,Targ,Rec,Rec%,Miscon,Dispos
0,Deportivo Alavés,es La Liga,32,41.1,17992,2141,6134,8850,3878,563,16042,250,466,53.6,277,14,9417,53334,26549,12391,9724,78.5,458,386
1,Amiens SC,fr Ligue 1,27,45.5,15903,1660,4991,8238,3582,415,14501,283,476,59.5,317,20,9853,53381,28763,11622,9628,82.8,409,426
2,Angers SCO,fr Ligue 1,26,45.1,16741,1449,5353,8130,4353,594,15332,344,558,61.6,367,26,10806,62206,34355,12508,10616,84.9,386,358
3,Arsenal,eng Premier League,29,54.0,24196,2929,8623,11508,5691,931,22390,428,711,60.2,455,34,16309,97513,55374,18851,16349,86.7,448,451
4,Aston Villa,eng Premier League,28,43.9,19826,2772,7317,8474,5327,780,17901,368,584,63.0,404,25,11555,74828,39599,14087,11530,81.8,409,428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,SV Werder Bremen,de Bundesliga,30,48.8,20863,2519,7265,10087,4783,704,19078,278,487,57.1,296,17,12938,73123,39211,15464,12978,83.9,453,403
97,West Bromwich Albion,eng Premier League,17,29.7,1336,219,554,565,282,36,1214,19,36,52.8,20,1,681,4004,2097,824,706,85.7,31,40
98,West Ham United,eng Premier League,29,44.0,20825,2288,6477,9965,5580,823,19019,406,645,62.9,429,23,12519,76114,39304,15315,12388,80.9,399,408
99,VfL Wolfsburg,de Bundesliga,26,48.3,19911,2102,6576,9616,4946,787,18132,388,617,62.9,414,30,11921,71694,40133,14309,11716,81.9,469,469


In [39]:

relegated = ['Bournemouth','Watford','Norwich City']
for i in range(len(gk_adv)):
  if gk_adv.loc[i,'Squad'] in relegated:
    gk_adv.loc[i,'Comp'] = 'eng Championship'
  gk_adv.loc[i,'Squad'] = pos_to_df[gk_adv.loc[i,'Squad']]
gk_adv

,Squad,Comp,# Pl,90s,GA,PKA,FK,CK,OG,PSxG,PSxG/SoT,PSxG+/-,/90,Cmp,Att,Cmp%,Att.1,Thr,Launch%,AvgLen,Att.2,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,Deportivo Alavés,es La Liga,3,38.0,59,12,3,3,0,57.0,0.32,-2.0,-0.05,401,881,45.5,787,110,71.5,50.4,348,91.4,62.5,356,22,6.2,18,0.47,14.0
1,Amiens SC,fr Ligue 1,1,28.0,50,1,1,13,1,41.1,0.29,-7.9,-0.28,231,521,44.3,567,95,55.7,43.6,254,80.7,55.4,269,15,5.6,11,0.39,13.7
2,Angers SCO,fr Ligue 1,2,28.0,33,3,1,1,1,25.8,0.27,-6.2,-0.22,162,412,39.3,528,124,47.3,39.8,222,73.0,52.4,215,17,7.9,22,0.79,14.8
3,Arsenal,eng Premier League,2,38.0,48,7,0,9,0,58.0,0.27,10.0,0.26,128,433,29.6,971,220,29.7,32.7,312,46.5,41.4,353,23,6.5,21,0.55,13.3
4,Aston Villa,eng Premier League,4,38.0,67,5,1,10,1,59.8,0.32,-6.2,-0.16,279,723,38.6,850,170,55.2,44.4,343,74.1,55.5,391,26,6.6,19,0.50,13.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,SV Werder Bremen,de Bundesliga,2,34.0,69,1,1,13,3,50.1,0.28,-15.9,-0.47,254,598,42.5,929,170,45.9,39.3,305,56.4,44.8,375,14,3.7,24,0.71,14.3
97,West Bromwich Albion,eng Premier League,1,3.0,11,2,0,3,0,10.5,0.42,-0.5,-0.18,23,69,33.3,70,11,72.9,51.3,21,85.7,55.8,28,1,3.6,0,0.00,13.3
98,West Ham United,eng Premier League,4,38.0,62,5,2,7,3,58.7,0.30,-0.3,-0.01,326,815,40.0,776,129,70.1,50.4,292,92.8,64.5,372,16,4.3,12,0.32,11.7
99,VfL Wolfsburg,de Bundesliga,2,34.0,46,5,1,5,0,45.9,0.31,-0.1,0.00,281,619,45.4,1012,161,47.4,40.0,240,57.9,48.1,290,21,7.2,30,0.88,15.4


In [40]:
for i in range(len(shooting)):
  shooting.loc[i,'Squad'] = pos_to_df[shooting.loc[i,'Squad']]
shooting

,Squad,Comp,# Pl,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,Deportivo Alavés,es La Liga,32,34,299,99,33.1,7.87,2.61,0.09,0.28,10.0,6,6,36.9,32.4,0.11,-2.9,-4.4
1,Amiens SC,fr Ligue 1,27,31,229,77,33.6,8.18,2.75,0.12,0.36,13.0,3,5,26.8,22.9,0.10,4.2,5.1
2,Angers SCO,fr Ligue 1,26,27,367,112,30.5,13.11,4.00,0.07,0.23,16.0,1,1,27.1,26.4,0.07,-0.1,-0.4
3,Arsenal,eng Premier League,29,56,401,144,35.9,10.55,3.79,0.13,0.37,19.0,3,3,49.2,46.9,0.12,6.8,6.1
4,Aston Villa,eng Premier League,28,40,453,146,32.2,11.92,3.84,0.09,0.27,17.0,1,3,40.0,37.6,0.08,0.0,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,SV Werder Bremen,de Bundesliga,30,40,419,135,32.2,12.32,3.97,0.09,0.27,10.0,3,6,41.4,36.9,0.09,-1.4,0.1
97,West Bromwich Albion,eng Premier League,17,5,22,8,36.4,7.33,2.67,0.23,0.63,2.0,0,0,1.7,1.7,0.08,3.3,3.3
98,West Ham United,eng Premier League,29,49,414,157,37.9,10.89,4.13,0.11,0.29,13.0,4,4,46.2,43.2,0.11,2.8,1.8
99,VfL Wolfsburg,de Bundesliga,26,48,466,155,33.3,13.71,4.56,0.09,0.28,11.0,4,4,47.3,44.3,0.10,0.7,-0.3


In [41]:
for i in range(len(sca)):
  sca.loc[i,'Squad'] = pos_to_df[sca.loc[i,'Squad']]
sca

,Squad,Comp,# Pl,SCA,SCA90,PassLive,PassDead,Drib,Sh,Fld,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,OG
0,Deportivo Alavés,es La Liga,32,454,11.95,295,72,20,27,40,52,1.37,31,7,1,6,7,0
1,Amiens SC,fr Ligue 1,27,351,12.54,256,27,28,11,29,50,1.79,34,0,6,4,6,0
2,Angers SCO,fr Ligue 1,26,544,19.43,400,57,35,23,28,48,1.71,36,2,3,3,3,1
3,Arsenal,eng Premier League,29,615,16.18,436,57,50,36,36,78,2.05,57,4,4,9,4,0
4,Aston Villa,eng Premier League,28,747,19.66,549,80,36,27,54,67,1.76,46,7,5,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,SV Werder Bremen,de Bundesliga,30,647,19.03,467,62,42,42,32,68,2.00,41,5,4,11,5,2
97,West Bromwich Albion,eng Premier League,17,31,10.33,26,1,2,0,2,6,2.00,3,1,1,0,1,0
98,West Ham United,eng Premier League,29,653,17.18,471,73,40,28,41,82,2.16,54,7,6,6,9,0
99,VfL Wolfsburg,de Bundesliga,26,699,20.56,482,82,50,45,40,76,2.24,48,5,8,7,8,0


In [42]:
def score_predict(club1,club2):
  '''
  This function attempts what is called in Sports Analysis as the ultimate 
  goal. Predicting the outcome of a match based on statistics.

  This is the first attempt at a basic static prediction involving two teams
  isolated and pitted together purely on the basis of their overall.

  Based on the overall and the playing style of the club, the model tries to 
  make a prediction on the possession throughout the match. Based on the 
  possession available to the team, the model predicts the shots on target 
  based on staggered Shots Created Prediction, depending on that particular team's
  average Shots and Chances created. 

  However, shots on target is not equal to goals scored. Which is why the Goalkeeper
  and the advanced Goalkeeper statistics are paramount to the model. 

  PSxG, or the Post Shot Goals Scored is a statistic for goalkeepers that is 
  meant to measure the probability that the shot results in a goal after a shot
  has been made. 

  It is a metric that enables accurate measure of the ability of the Goalkeeper.

  Kepa Arrizabalaga, the misfiring keeper of Chelsea has an astonishing 0.35 
  PSxG/Shots on Target. Which in essence means that for a player shooting on 
  target against Kepa Arrizabalaga has an approximately 1/3rd of a chance of
  scoring. Try fixing that eh, Timo Werner?

  The final goals tally is then compared to that of the other team, and the result
  is given out. 
  '''
  m1 = club1.mid_ovr
  m2 = club2.mid_ovr
  d1 = club1.def_ovr
  d2 = club2.def_ovr
  a1 = club1.att_ovr
  a2 = club2.att_ovr
  s1 = (2*m1 + a1)/3
  s2 = (2*m2 + a2)/3
  #pos1 = 2*(m1 - m2) + (d1 - d2) + (a1 - a2)
  #pos2 = 2*(m2 - m1) + (d2 - d1) + (a2 - a1)
  #perc1 = s1*100/(s1+s2)
  #perc2 = s2*100/(s1+s2)
  pos1 = (s1-s2)*2
  pos2 = (s2-s1)*2
  per1 = per2 = 50
  per1 = per1 + pos1
  per2 = per2 + pos2
  pos_ratio = s1/s2
  print("Possession Stats : ")
  '''
  if(s1>s2):
    if (s1-s2 > 5):
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
    else :
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,perc1,perc2,club2.team_name))
    pos_ratio = s1/s2
  else:  
    if (s2-s1 > 5):
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
    else :
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,perc1,perc2,club2.team_name))
    pos_ratio = s2/s1
  '''
  print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
  shots_created_1 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per1/50) * ( ( (m1+a1)/2 )/d2 ) )  
  shots_created_2 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per2/50) * ( ( (m2+a2)/2 )/d1 ) )
  #print(' {} {} '.format(s1,s2))
  
  print('Shots Created by {} : {} '.format(club1.team_name,shots_created_1))
  print('Shots Created by {} : {} '.format(club2.team_name,shots_created_2))

  sot_1 = int(shooting[shooting['Squad'] == club1.team_name]['SoT%'].values[0] * (shots_created_1/100) * ( a1 / d2) )
  sot_2 = int(shooting[shooting['Squad'] == club2.team_name]['SoT%'].values[0] * (shots_created_2/100) * ( a2 / d1) )
  print('Shots on target by {} : {} '.format(club1.team_name,sot_1))
  print('Shots on target by {} : {} '.format(club2.team_name,sot_2))

  ex_g1 = int(gk_adv[gk_adv['Squad'] == club2.team_name]['PSxG/SoT'].values[0]*sot_1)
  ex_g2 = int(gk_adv[gk_adv['Squad'] == club1.team_name]['PSxG/SoT'].values[0]*sot_2)

  x_g1 = int(shooting[shooting['Squad'] == club1.team_name]['G/SoT'].values[0]*sot_1)
  x_g2 = int(shooting[shooting['Squad'] == club2.team_name]['G/SoT'].values[0]*sot_2)
  print("Predicted Scoreline :")
  print(" {} {} : {} {}".format(club1.team_name,ex_g1,ex_g2,club2.team_name))
  #print(" {} {} : {} {}".format(club1.team_name,x_g1,x_g2,club2.team_name))

In [43]:
MUN = club('Manchester United')
BVB09 = club('Borussia Dortmund')
a = BVB09.get_ovr(BVB09.goalkeep,BVB09.defense,BVB09.midfield,BVB09.attack,BVB09.starting_eleven,BVB09.sub_5,['D','D','D'])
b = MUN.get_ovr(MUN.goalkeep,MUN.defense,MUN.midfield,MUN.attack,MUN.starting_eleven,MUN.sub_5,['D','D','D'])

score_predict(BVB09,MUN)
"""
As per the function, a match between BVB09 and MUFC would result in a 1-1 draw,
which would see Dortmund come away with more possession, but are let down by
Roman Burki who has an astonishingly high 0.34 PSxG/SoT
"""

Possession Stats : 
Borussia Dortmund  55.333 : 44.667 Manchester United
Shots Created by Borussia Dortmund : 14 
Shots Created by Manchester United : 11 
Shots on target by Borussia Dortmund : 6 
Shots on target by Manchester United : 4 
Predicted Scoreline :
 Borussia Dortmund 1 : 1 Manchester United


In [44]:
def form_variate_score_predict(club1,club2):
  """
  While the function above predicts the most likely outcome, football is rarely
  a game of static prediction. More often than not there are slips in defense, 
  and astonishing results like Leeds schooling Liverpool (for a half anyway)

  This function aims to bring both the form of the teams into the prediction, 
  and random pure chance, coupled with the Home side advantage, to produce a
  less statistically modelled but more likely prediction.

  No two instances of this function on two teams is exactly the same.

  We use the form class variable to induce an integer variable called form to
  affect the outcome. For example, a Norwich City team coming off of three 
  consecutive losses is bound to lose to a Manchester City team on a roll. But
  reverse the forms and we have a game in hand, complete with Todd Cantwell dancing
  on the non existent City fans. 
  """
  form1 = club1.form
  form2 = club2.form
  f1 = 1
  f2 = 1
  if len(form1) != 0:
    for i in range(len(form1)):
     if form1[i] == 'W':
      f1 = (f1 + 0.3)       # adding a bonus of 0.3 for a win
     if form2[i] == 'D':
      f1 = (f1 + 0.1)       # and a 0.1 for a draw
  
  if len(form1) != 0:
    f1 = f1/len(form1)
  if len(form2) != 0:
    for i in range(len(form2)):
      if form2[i] == 'W':
        f2 = (f2 + 0.3)
      if form2[i] == 'D':
        f2 = (f2 + 0.1)
  
  if len(form2) != 0:
    f2 = f2/len(form2)
  #print(' {} {} '.format(f1,f2))

  m1 = club1.mid_ovr
  m2 = club2.mid_ovr
  d1 = club1.def_ovr
  d2 = club2.def_ovr
  a1 = club1.att_ovr
  a2 = club2.att_ovr
  rand1 = np.random.uniform(0.5,1.5)
  rand2 = np.random.uniform(0.5,1.5)
  s1 = (2*m1 + a1)/3        # obtaining an overall that combines the midfield and attack overalls to estimate possession
  s2 = (2*m2 + a2)/3
  #pos1 = 2*(m1 - m2) + (d1 - d2) + (a1 - a2)
  #pos2 = 2*(m2 - m1) + (d2 - d1) + (a2 - a1)
  #perc1 = s1*100/(s1+s2)
  #perc2 = s2*100/(s1+s2)
  pos1 = (s1-s2)*2
  pos2 = (s2-s1)*2
  per1 = per2 = 50
  per1 = per1 + pos1
  per2 = per2 + pos2
  pos_ratio = s1/s2
  #print('{} {}'.format(rand1,rand2))
  print("Possession Stats : ")
  '''
  if(s1>s2):
    if (s1-s2 > 5):
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
    else :
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,perc1,perc2,club2.team_name))
    pos_ratio = s1/s2
  else:  
    if (s2-s1 > 5):
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
    else :
      print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,perc1,perc2,club2.team_name))
    pos_ratio = s2/s1
  '''
  ran1 = np.random.uniform(0.8,1.3)
  per1 = per1*ran1
  #per1 = per1*
  print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,100 - per1,club2.team_name))
  shots_created_1 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per1/50) * ( ( (m1+a1)/2 )/d2 ) * (f1/f2)) #''' * (rand1) '''  
  shots_created_2 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per2/50) * ( ( (m2+a2)/2 )/d1 ) * (f1/f2))  #'''* (rand2) ''')
  #print(' {} {} '.format(s1,s2))
  
  print('Chances Created by {} : {} '.format(club1.team_name,shots_created_1))
  print('Chances Created by {} : {} '.format(club2.team_name,shots_created_2))

  sot_1 = int(shooting[shooting['Squad'] == club1.team_name]['SoT%'].values[0] * (shots_created_1/100)  * (rand1) )    #* ( a1 / d2)
  sot_2 = int(shooting[shooting['Squad'] == club2.team_name]['SoT%'].values[0] * (shots_created_2/100)  * (rand2) )    #* ( a2 / d1) 
  print('Shots on target by {} : {} '.format(club1.team_name,sot_1))
  print('Shots on target by {} : {} '.format(club2.team_name,sot_2))

  ex_g1 = min(int(gk_adv[gk_adv['Squad'] == club2.team_name]['PSxG/SoT'].values[0]*sot_1 ),8)#* rand1)
  ex_g2 = min(int(gk_adv[gk_adv['Squad'] == club1.team_name]['PSxG/SoT'].values[0]*sot_2 ),8)#* rand2)

  x_g1 = int(shooting[shooting['Squad'] == club1.team_name]['G/SoT'].values[0]*sot_1)
  x_g2 = int(shooting[shooting['Squad'] == club2.team_name]['G/SoT'].values[0]*sot_2)
  print("Predicted Scoreline :")
  print(" {} {} : {} {}".format(club1.team_name,ex_g1,ex_g2,club2.team_name))
  if ex_g1>ex_g2:
    print('{} Wins'.format(club1.team_name))
  elif ex_g2>ex_g1:
    print('{} Wins'.format(club2.team_name))
  else:
    print('Draw')
  return (ex_g1,ex_g2)

In [51]:

BVB09 = club('Manchester City')
MUN = club('Manchester United')
BVB09.get_ovr(BVB09.goalkeep,BVB09.defense,BVB09.midfield,BVB09.attack,BVB09.starting_eleven,BVB09.sub_5,[])
MUN.get_ovr(MUN.goalkeep,MUN.defense,MUN.midfield,MUN.attack,MUN.starting_eleven,MUN.sub_5,[])

(h,a) = form_variate_score_predict(BVB09,MUN)

Possession Stats : 
Manchester City  48.939 : 51.061 Manchester United
Chances Created by Manchester City : 19 
Chances Created by Manchester United : 14 
Shots on target by Manchester City : 3 
Shots on target by Manchester United : 5 
Predicted Scoreline :
 Manchester City 0 : 1 Manchester United
Manchester United Wins


In [52]:
lt = pd.read_csv('league table.csv',error_bad_lines=False,encoding = 'ISO-8859-1')
# Importing the league table to modify and use
lt

,Rk,Squad,MP,W,D,L,GF,GA,GDiff,Pts,Last 5,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,5,Arsenal,3,2,0,1,6,4,2,6,W W L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Aston Villa,2,2,0,0,4,0,4,6,W W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12,Brighton,3,1,0,2,6,6,0,3,L W L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,Burnley,2,0,0,2,2,5,-3,0,L L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Chelsea,3,1,1,1,6,6,0,4,W L D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Crystal Palace,3,2,0,1,5,3,2,6,W W L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,Everton,3,3,0,0,8,3,5,9,W W W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20,Fulham,3,0,0,3,3,10,-7,0,L L L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,Leeds United,3,2,0,1,8,7,1,6,L W W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,Leicester City,3,3,0,0,12,4,8,9,W W W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Clearing existing data to get a fresh league table

lt = lt.iloc[:,:11]
lt.columns = ['Rk'	,'Squad'	,'MP'	,'W',	'D'	,'L',	'GF',	'GA',	'GDiff'	,'Pts'	,'Last 3']
table_cols = ['MP'	,'W',	'D'	,'L',	'GF',	'GA',	'GDiff'	,'Pts']
for col in table_cols:
  lt[col] = np.zeros_like(lt[col])
lt['Last 3'] = [[] for x in lt['Last 3']]
lt['Rk'] = np.arange(1,21)
lt

,Rk,Squad,MP,W,D,L,GF,GA,GDiff,Pts,Last 3
0,1,Arsenal,0,0,0,0,0,0,0,0,[]
1,2,Aston Villa,0,0,0,0,0,0,0,0,[]
2,3,Brighton,0,0,0,0,0,0,0,0,[]
3,4,Burnley,0,0,0,0,0,0,0,0,[]
4,5,Chelsea,0,0,0,0,0,0,0,0,[]
5,6,Crystal Palace,0,0,0,0,0,0,0,0,[]
6,7,Everton,0,0,0,0,0,0,0,0,[]
7,8,Fulham,0,0,0,0,0,0,0,0,[]
8,9,Leeds United,0,0,0,0,0,0,0,0,[]
9,10,Leicester City,0,0,0,0,0,0,0,0,[]


In [54]:
# Changing the league names to official FIFA names 
for i in range(len(lt)):
  lt.loc[i,'Squad'] = pos_to_df[lt.loc[i,'Squad']]
lt

,Rk,Squad,MP,W,D,L,GF,GA,GDiff,Pts,Last 3
0,1,Arsenal,0,0,0,0,0,0,0,0,[]
1,2,Aston Villa,0,0,0,0,0,0,0,0,[]
2,3,Brighton & Hove Albion,0,0,0,0,0,0,0,0,[]
3,4,Burnley,0,0,0,0,0,0,0,0,[]
4,5,Chelsea,0,0,0,0,0,0,0,0,[]
5,6,Crystal Palace,0,0,0,0,0,0,0,0,[]
6,7,Everton,0,0,0,0,0,0,0,0,[]
7,8,Fulham,0,0,0,0,0,0,0,0,[]
8,9,Leeds United,0,0,0,0,0,0,0,0,[]
9,10,Leicester City,0,0,0,0,0,0,0,0,[]


In [55]:
print("Teams in the Premier League : ")
x = gk_adv[gk_adv['Comp'] == 'eng Premier League']['Squad'].values
print(x)
first_10 = x[:10]
last_10 = x[10:]

Teams in the Premier League : 
['Arsenal' 'Aston Villa' 'Brighton & Hove Albion' 'Burnley' 'Chelsea'
 'Crystal Palace' 'Everton' 'Fulham' 'Leeds United' 'Leicester City'
 'Liverpool' 'Manchester City' 'Manchester United' 'Newcastle United'
 'Sheffield United' 'Southampton' 'Tottenham Hotspur'
 'West Bromwich Albion' 'West Ham United' 'Wolverhampton Wanderers']


In [58]:
# Reading the fixture list for the Premier League Season 2020-21
fixture_list = pd.read_csv('fixture_list.csv',error_bad_lines=False,encoding='ISO-8859-1')
fixture_list = fixture_list.iloc[:,:3]
fixture_list

,Wk,Home,Away
0,1.0,Burnley,Manchester Utd
1,1.0,Manchester City,Aston Villa
2,1.0,Fulham,Arsenal
3,1.0,Crystal Palace,Southampton
4,1.0,Liverpool,Leeds United
...,...,...,...
412,38.0,Fulham,Newcastle Utd
413,38.0,West Ham,Southampton
414,38.0,Arsenal,Brighton
415,38.0,Sheffield Utd,Burnley


In [59]:
for i in range(10,407,11):
  fixture_list = fixture_list.drop(i,axis = 0)
fixture_list

,Wk,Home,Away
0,1.0,Burnley,Manchester Utd
1,1.0,Manchester City,Aston Villa
2,1.0,Fulham,Arsenal
3,1.0,Crystal Palace,Southampton
4,1.0,Liverpool,Leeds United
...,...,...,...
412,38.0,Fulham,Newcastle Utd
413,38.0,West Ham,Southampton
414,38.0,Arsenal,Brighton
415,38.0,Sheffield Utd,Burnley


In [60]:
fixture_list = fixture_list.reset_index(drop=True)
fixture_list

,Wk,Home,Away
0,1.0,Burnley,Manchester Utd
1,1.0,Manchester City,Aston Villa
2,1.0,Fulham,Arsenal
3,1.0,Crystal Palace,Southampton
4,1.0,Liverpool,Leeds United
...,...,...,...
375,38.0,Fulham,Newcastle Utd
376,38.0,West Ham,Southampton
377,38.0,Arsenal,Brighton
378,38.0,Sheffield Utd,Burnley


In [61]:
# Changing the fixture list club names to FIFA format
for i in range(len(fixture_list)):
  fixture_list.loc[i,'Home'] = pos_to_df[fixture_list.loc[i,'Home']]
  fixture_list.loc[i,'Away'] = pos_to_df[fixture_list.loc[i,'Away']]

In [62]:
fixture_list

,Wk,Home,Away
0,1.0,Burnley,Manchester United
1,1.0,Manchester City,Aston Villa
2,1.0,Fulham,Arsenal
3,1.0,Crystal Palace,Southampton
4,1.0,Liverpool,Leeds United
...,...,...,...
375,38.0,Fulham,Newcastle United
376,38.0,West Ham United,Southampton
377,38.0,Arsenal,Brighton & Hove Albion
378,38.0,Sheffield United,Burnley


In [63]:
fixture_by_gameweek = {}
for i in range(1,39):
  fixture_by_gameweek['W'+str(i)] = fixture_list[fixture_list['Wk'] == i]
fix_by_gk_2 = {}
for i in range(1,5):
  fix_by_gk_2['W'+str(i)] = fixture_list[fixture_list['Wk'] == i]

In [64]:
# Score Prediction function without print statements
def form_variate_func_predict(club1,club2):
  
  form1 = club1.form
  form2 = club2.form
  #print(form1)
  #print(form2)
  f1 = 1
  f2 = 1
  
  if len(form1) != 0:
    for i in range(len(form1)):
     if form1[i] == 'W':
      f1 = (f1 + 0.2)
     if form2[i] == 'D':
      f1 = (f1 + 0.15)
  
  if len(form1) != 0:
    f1 = f1/len(form1)
  if len(form2) != 0:
    for i in range(len(form2)):
      if form2[i] == 'W':
        f2 = (f2 + 0.2)
      if form2[i] == 'D':
        f2 = (f2 + 0.15)
  
  if len(form2) != 0:
    f2 = f2/len(form2)
  #print(' {} {} '.format(f1,f2))
  m1 = club1.mid_ovr
  m2 = club2.mid_ovr
  d1 = club1.def_ovr
  d2 = club2.def_ovr
  a1 = club1.att_ovr
  a2 = club2.att_ovr
  rand1 = np.random.uniform(0.5,2.1)
  rand2 = np.random.uniform(0.5,1.9)
  s1 = (2*m1 + a1)/3
  s2 = (2*m2 + a2)/3
  #pos1 = 2*(m1 - m2) + (d1 - d2) + (a1 - a2)
  #pos2 = 2*(m2 - m1) + (d2 - d1) + (a2 - a1)
  #perc1 = s1*100/(s1+s2)
  #perc2 = s2*100/(s1+s2)
  pos1 = (s1-s2)*2
  pos2 = (s2-s1)*2
  per1 = per2 = 50
  per1 = per1 + pos1
  per2 = per2 + pos2
  pos_ratio = s1/s2
  #print('{} {}'.format(rand1,rand2))
  #print("Possession Stats : ")

  #print("{}  {:.3f} : {:.3f} {}".format(club1.team_name,per1,per2,club2.team_name))
  shots_created_1 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per1/50) * ( ( (m1+a1)/2 )/d2 ) * (f1/f2))  
  shots_created_2 = int(shooting[shooting['Squad'] == club1.team_name]['Sh/90'].values[0]* (per2/50) * ( ( (m2+a2)/2 )/d1 ) * (f1/f2)) 
  #print(' {} {} '.format(s1,s2))
  
  #print('Chances Created by {} : {} '.format(club1.team_name,shots_created_1))
  #print('Chances Created by {} : {} '.format(club2.team_name,shots_created_2))

  sot_1 = int(shooting[shooting['Squad'] == club1.team_name]['SoT%'].values[0] * (shots_created_1/100)  * (rand1) )    #* ( a1 / d2)
  sot_2 = int(shooting[shooting['Squad'] == club2.team_name]['SoT%'].values[0] * (shots_created_2/100)  * (rand2) )    #* ( a2 / d1) 
  #print('Shots on target by {} : {} '.format(club1.team_name,sot_1))
  #print('Shots on target by {} : {} '.format(club2.team_name,sot_2))

  ex_g1 = min(int(gk_adv[gk_adv['Squad'] == club2.team_name]['PSxG/SoT'].values[0]*sot_1 ),8)#* rand1)
  ex_g2 = min(int(gk_adv[gk_adv['Squad'] == club1.team_name]['PSxG/SoT'].values[0]*sot_2 ),8)#* rand2)

  x_g1 = int(shooting[shooting['Squad'] == club1.team_name]['G/SoT'].values[0]*sot_1)
  x_g2 = int(shooting[shooting['Squad'] == club2.team_name]['G/SoT'].values[0]*sot_2)
  #print("Predicted Scoreline :")
  print(" {} {} : {} {}".format(club1.team_name,ex_g1,ex_g2,club2.team_name))
  '''
  if ex_g1>ex_g2:
    print('{} Wins'.format(club1.team_name))
  elif ex_g2>ex_g1:
    print('{} Wins'.format(club2.team_name))
  else:
    print('Draw')
  '''
  return (ex_g1,ex_g2)

In [65]:
# Let us try simming the first 4 matchweeks to get an idea of whether our function works
for col in table_cols:
  lt[col] = np.zeros_like(lt[col])
lt['Last 3'] = [[] for x in lt['Last 3']]
lt['Rk'] = np.arange(1,21)
k = 0
for a in fix_by_gk_2:
  
  fix_list = fixture_by_gameweek[a]
  fix_list = fix_list.reset_index(drop = True)
  #print(fix_list)
  lt['MP'] += 1
  for i in range(len(fix_list)):  
    H_num = lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0]
    A_num = lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0]
    H = club(fix_list.loc[i,'Home'])
    A = club(fix_list.loc[i,'Away'])
    H_form = lt[lt['Squad'] == fix_list.loc[i,'Away']]['Last 3'].values[0]
    A_form = lt[lt['Squad'] == fix_list.loc[i,'Away']]['Last 3'].values[0]
    H.get_ovr(H.goalkeep,H.defense,H.midfield,H.attack,H.starting_eleven,H.sub_5,H_form)
    A.get_ovr(A.goalkeep,A.defense,A.midfield,A.attack,A.starting_eleven,A.sub_5,A_form)
    
    #print('Hello')
    (g1,g2) = form_variate_func_predict(H,A)
    #print(g1,g2)
    if g1>g2:
      #print('Home win loop')
      lt.loc[H_num,'W'] += 1
      lt.loc[A_num,'L'] += 1
      lt.loc[H_num,'Pts'] += 3
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('L')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('W')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    elif g2>g1:
      #print('Away win loop')
      lt.loc[A_num,'W'] += 1
      lt.loc[H_num,'L'] += 1
      lt.loc[A_num,'Pts'] += 3
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('W')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('L')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    else:
      #print('Draw loop')
      lt.loc[H_num,'D'] += 1
      lt.loc[A_num,'D'] += 1
      lt.loc[H_num,'Pts'] += 1
      lt.loc[A_num,'Pts'] += 1
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('D')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('D')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    lt.loc[H_num,'GF'] = lt.loc[H_num,'GF'] + g1
    lt.loc[A_num,'GF'] = lt.loc[A_num,'GF'] + g2
    lt.loc[A_num,'GA'] = lt.loc[A_num,'GA'] + g1
    lt.loc[H_num,'GA'] = lt.loc[H_num,'GA'] + g2
    lt.loc[H_num,'GDiff'] = lt.loc[H_num,'GDiff'] + g1 - g2
    lt.loc[A_num,'GDiff'] = lt.loc[A_num,'GDiff'] + g2 - g1
  disp_lt = lt.sort_values(by = ['Pts','GDiff'],ascending=False)
  disp_lt['Rk'] = np.arange(1,21)
  print(disp_lt)
  k += 1


 Burnley 0 : 0 Manchester United
 Manchester City 2 : 1 Aston Villa
 Fulham 1 : 2 Arsenal
 Crystal Palace 1 : 0 Southampton
 Liverpool 4 : 0 Leeds United
 West Ham United 2 : 1 Newcastle United
 West Bromwich Albion 0 : 0 Leicester City
 Tottenham Hotspur 0 : 0 Everton
 Sheffield United 0 : 1 Wolverhampton Wanderers
 Brighton & Hove Albion 1 : 2 Chelsea
    Rk                    Squad  MP  W  D  L  GF  GA  GDiff  Pts Last 3
10   1                Liverpool   1  1  0  0   4   0      4    3    [W]
0    2                  Arsenal   1  1  0  0   2   1      1    3    [W]
4    3                  Chelsea   1  1  0  0   2   1      1    3    [W]
5    4           Crystal Palace   1  1  0  0   1   0      1    3    [W]
11   5          Manchester City   1  1  0  0   2   1      1    3    [W]
18   6          West Ham United   1  1  0  0   2   1      1    3    [W]
19   7  Wolverhampton Wanderers   1  1  0  0   1   0      1    3    [W]
3    8                  Burnley   1  0  1  0   0   0      0    1    

In [69]:
# Now that we can see that the sim works for the first 4 matchweeks, let us 
# now sim the entire league fixture list. 

# Keep in mind that no two instances of the simulator will be same, due to the
# added spontaneity. 
for col in table_cols:
  lt[col] = np.zeros_like(lt[col])
lt['Last 3'] = [[] for x in lt['Last 3']]
lt['Rk'] = np.arange(1,21)
k = 0
for a in fixture_by_gameweek:
  
  fix_list = fixture_by_gameweek[a]
  fix_list = fix_list.reset_index(drop = True)
  #print(fix_list)
  lt['MP'] += 1
  for i in range(len(fix_list)):  
    H_num = lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0]
    A_num = lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0]
    H = club(fix_list.loc[i,'Home'])
    A = club(fix_list.loc[i,'Away'])
    H_form = lt[lt['Squad'] == fix_list.loc[i,'Away']]['Last 3'].values[0]
    A_form = lt[lt['Squad'] == fix_list.loc[i,'Away']]['Last 3'].values[0]
    H.get_ovr(H.goalkeep,H.defense,H.midfield,H.attack,H.starting_eleven,H.sub_5,H_form)
    A.get_ovr(A.goalkeep,A.defense,A.midfield,A.attack,A.starting_eleven,A.sub_5,A_form)
    
    #print('Hello')
    (g1,g2) = form_variate_func_predict(H,A)
    #print(g1,g2)
    if g1>g2:
      #print('Home win loop')
      lt.loc[H_num,'W'] += 1
      lt.loc[A_num,'L'] += 1
      lt.loc[H_num,'Pts'] += 3
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('L')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('W')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    elif g2>g1:
      #print('Away win loop')
      lt.loc[A_num,'W'] += 1
      lt.loc[H_num,'L'] += 1
      lt.loc[A_num,'Pts'] += 3
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('W')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('L')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    else:
      #print('Draw loop')
      lt.loc[H_num,'D'] += 1
      lt.loc[A_num,'D'] += 1
      lt.loc[H_num,'Pts'] += 1
      lt.loc[A_num,'Pts'] += 1
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].append('D')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Away']].index.values[0],'Last 3'].pop(0)
      lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].append('D')
      if (len(lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3']) > 3):
        lt.loc[lt[lt['Squad'] == fix_list.loc[i,'Home']].index.values[0],'Last 3'].pop(0)
    lt.loc[H_num,'GF'] = lt.loc[H_num,'GF'] + g1
    lt.loc[A_num,'GF'] = lt.loc[A_num,'GF'] + g2
    lt.loc[A_num,'GA'] = lt.loc[A_num,'GA'] + g1
    lt.loc[H_num,'GA'] = lt.loc[H_num,'GA'] + g2
    lt.loc[H_num,'GDiff'] = lt.loc[H_num,'GDiff'] + g1 - g2
    lt.loc[A_num,'GDiff'] = lt.loc[A_num,'GDiff'] + g2 - g1
  disp_lt = lt.sort_values(by = ['Pts','GDiff'],ascending=False)
  disp_lt['Rk'] = np.arange(1,21)
  print(disp_lt)
  k += 1


 Burnley 0 : 1 Manchester United
 Manchester City 6 : 1 Aston Villa
 Fulham 0 : 3 Arsenal
 Crystal Palace 1 : 1 Southampton
 Liverpool 1 : 0 Leeds United
 West Ham United 0 : 0 Newcastle United
 West Bromwich Albion 0 : 1 Leicester City
 Tottenham Hotspur 2 : 0 Everton
 Sheffield United 0 : 1 Wolverhampton Wanderers
 Brighton & Hove Albion 1 : 1 Chelsea
    Rk                    Squad  MP  W  D  L  GF  GA  GDiff  Pts Last 3
11   1          Manchester City   1  1  0  0   6   1      5    3    [W]
0    2                  Arsenal   1  1  0  0   3   0      3    3    [W]
16   3        Tottenham Hotspur   1  1  0  0   2   0      2    3    [W]
9    4           Leicester City   1  1  0  0   1   0      1    3    [W]
10   5                Liverpool   1  1  0  0   1   0      1    3    [W]
12   6        Manchester United   1  1  0  0   1   0      1    3    [W]
19   7  Wolverhampton Wanderers   1  1  0  0   1   0      1    3    [W]
2    8   Brighton & Hove Albion   1  0  1  0   1   1      0    1    

In [ ]:
"""
    And there you have it!
    My first attempt at Sports Analytics and the league for the season 2020-21
    has been successfully simulated, though only time will tell how accurate 
    it really is. 

    To summarise the season, an aggressive follow-up on last season's heroics 
    see the Liverpool clinch the title with another 99 points season, losing 
    only once to a spirited Chelsea on the second matchday. Manchester City 
    are unable to keep the same tempo that Liverpool keep and fall to another
    second place finish. Tottenham Hotspurs are surprise candidates coming in
    3rd, and to complete a North London CL places coup Arsenal follows in at 
    4th, with United clinching a 5th place (mainly due to poor recruitment if
    you ask me)

    Unable to follow a fantastic first season showing, Sheffield United fall 
    to the relegation places alongside the newly promoted WBA and Fulham. No
    surprises there.

    A strong start to the season enables Everton to finish in the top half while
    a strong start not maintained by Bielsa's men prompts them to finish 16th, 
    while Aston Villa once again survive by one place. 

    Thank You!
"""

In [ ]:
"""
LEE = club('Leeds United')
LEE.get_team_data()
LEE.team_data.loc[:,('short_name','player_positions','overall','team_jersey_number')]
LEE.get_ovr(
    {
     19:'gk',
     6:'cb',
     28:'cb',
     3:'lb',
     2:'rb',
     43:'cm',
     23:'cdm',
     4:'cm',
     19:'lm',
     17:'rm',
     9:'st'    
    },
    {
     22:'sub',
     28:'sub',
     10:'sub',
     11:'sub',
     14:'sub'
    }
)
"""

In [ ]:
"""
MUN.get_ovr(
    {
        1:'gk',
        2:'cb',
        5:'cb',
        29:'rb',
        23:'lb',
        6:'cm',
        10:'lm',
        8:'rm',
        7:'lw',
        9:'st',
        14:'rw'
    },
    {
        31:'Sub',
        3:'Sub',
        22:'Sub',
        12:'Sub',
        17:'Sub',
    }
)
print(MUN.lineup)
print(MUN.ovr)
"""

In [ ]:
#get_team_data('Borussia Dortmund')
dort = {
    1:'gk',                             #Burki
    25:'cb',                            #Akanji
    16:'cb',                            #Hummels
    5:'lb',                             #Pisczek
    26:'rb',                            #Hakimi
    7:'lm',                             #Sancho
    6:'cdm',
    11:'cam',
    28:'cdm',
    23:'rm',
    9:'st'
}
do_sub = {
    35:'Sub',
    13:'Sub',
    33:'Sub',
    2:'Sub',
    8:'Sub'
    
}

In [ ]:
'''def head2head(club1,club2):

  club1_ovr = club1.ovr/11
  club2_ovr = club2.ovr/11
  club1_ch = club1_ovr/(club1_ovr+club2_ovr)
  club1_ch = club2_ch = draw_ch =  33.3333
  club1_ch = club1_ch + (club1_ovr - club2_ovr)/3
  club2_ch = club2_ch + (club2_ovr - club1_ovr)/3
  draw_ch = draw_ch #- (abs(club1_ovr - club2_ovr))/3
  #print('{} : {:.4f} Draw : {:.4f} {} : {:.4f}'.format(club1.team_name,club1_ch,draw_ch,club2.team_name,club2_ch))
  print('Difference : {}'.format(abs(club1_ovr - club2_ovr)))
  #print('Sum : {}'.format(club1_ch+club2_ch+draw_ch))'''

In [ ]:
"""MUN = club('Manchester City')
BVB09 = club('FC Bayern München')
a = BVB09.get_ovr(BVB09.goalkeep,BVB09.defense,BVB09.midfield,BVB09.attack,BVB09.starting_eleven,BVB09.sub_5,['W','W','L'])

b = MUN.get_ovr(MUN.goalkeep,MUN.defense,MUN.midfield,MUN.attack,MUN.starting_eleven,MUN.sub_5,['W','W','L'])
#poss_predict(BVB09,MUN)"""